# Weapon–Target Assignment via Piecewise Linearization and CPLEX

Este notebook muestra paso a paso cómo resolver el modelo WTA (Ecuación 3.1)

---

## 1. Modelo canónico

**Variables de decisión**  
- $x_{ij}\in\mathbb{Z}_+$: número de armas de tipo $i$ asignadas al blanco $j$.

**Parámetros**  
- $w_i$: inventario disponible de armas tipo $i$.  
- $V_j$: valor de destrucción del blanco $j$.  
- $p_{ij}$: probabilidad de impacto de un arma tipo $i$ sobre el blanco $j$.  
- Definimos $q_{ij} = 1 - p_{ij}$.

**Función objetivo**  
> Minimizar la suma de supervivencias ponderadas:  
>
> $$
> \min_{x\in\mathbb{Z}_+}\;F(x)
> \;=\;
> \sum_{j=1}^n V_j \,\prod_{i=1}^m (1 - p_{ij})^{\,x_{ij}}
> \;=\;
> \sum_{j=1}^n V_j \exp\!\Bigl(\sum_{i=1}^m x_{ij}\ln q_{ij}\Bigr).
> $$

**Restricciones**  
> Inventario de armas:
> $$
> \sum_{j=1}^n x_{ij} \;\le\; w_i
> \quad
> \forall\,i=1,\dots,m
> $$
>
> Dominio entero:
> $$
> x_{ij}\in\mathbb{Z}_+,
> \quad
> \forall\,i=1,\dots,m,\;j=1,\dots,n.
> $$

---

## 2. Linealización _Piecewise Linear_ (PWL)

Para cada blanco $j$, definimos la variable auxiliar  
$
y_j = \sum_{i=1}^m x_{ij}\,\ln q_{ij},
$
y aproximamos  
$
f_j = V_j\,e^{y_j}
$
mediante segmentos lineales:

1. **Bounds de** $y_j$  
   $
     \underline y_j = \sum_{i=1}^m w_i\,\ln(q_{ij}),
     \quad
     \overline y_j = 0.
   $

2. **Puntos de ruptura**  
   Para $k=0,1,\dots,K$:
   $
     \xi_{j,k}
     = \underline y_j + \frac{k}{K}\bigl(\overline y_j - \underline y_j\bigr),
     \quad
     f_{j,k} = V_j\,e^{\,\xi_{j,k}}.
   $

3. **Variables PWL**  
   $\lambda_{j,k}\ge0$, con
   $$
     \sum_{k=0}^K \lambda_{j,k} = 1,\quad
     y_j = \sum_{k=0}^K \xi_{j,k}\,\lambda_{j,k},\quad
     f_j = \sum_{k=0}^K f_{j,k}\,\lambda_{j,k}.
   $$

---

In [9]:
import math
import random
import pulp

def solve_wta_pwl(
    w,        # lista de inventarios w[i] para cada tipo i
    n,        # número de blancos
    K=8,      # número de tramos PWL
    seed=218   # semilla para reproducibilidad
):
    random.seed(seed)
    m = len(w)

    # 1) Generar aleatoriamente p_{ij}∈[0.5,0.8] y V_j∈[5,10]
    p = [[random.uniform(0.5, 0.8) for j in range(n)] for i in range(m)]
    V = [random.uniform(5, 10)   for j in range(n)]

    # 2) Pre-cálculos
    q     = [[1 - p[i][j] for j in range(n)] for i in range(m)]
    y_min = [sum(w[i]*math.log(q[i][j]) for i in range(m)) for j in range(n)]
    breaks = {
        j: [y_min[j] + k*(0.0 - y_min[j])/K for k in range(K+1)]
        for j in range(n)
    }
    f_vals = {
        j: [V[j]*math.exp(y) for y in breaks[j]]
        for j in range(n)
    }

    # 3) Construir el modelo PWL en PuLP
    model = pulp.LpProblem("WTA_PWL", pulp.LpMinimize)

    # Variables
    x   = pulp.LpVariable.dicts("x",   (range(m), range(n)), lowBound=0, cat="Integer")
    y   = { j: pulp.LpVariable(f"y_{j}", lowBound=y_min[j], upBound=0.0)
            for j in range(n) }
    f_j = { j: pulp.LpVariable(f"f_{j}", lowBound=min(f_vals[j]), upBound=max(f_vals[j]))
            for j in range(n) }
    lam = { (j,k): pulp.LpVariable(f"lam_{j}_{k}", lowBound=0)
            for j in range(n) for k in range(K+1) }

    # 4) Restricciones PWL + enlace
    for j in range(n):
        model += pulp.lpSum(lam[(j,k)] for k in range(K+1)) == 1
        model += y[j]   == pulp.lpSum(breaks[j][k]  * lam[(j,k)] for k in range(K+1))
        model += f_j[j] == pulp.lpSum(f_vals[j][k] * lam[(j,k)] for k in range(K+1))
        model += y[j]   == pulp.lpSum(x[i][j] * math.log(q[i][j]) for i in range(m))

    # 5) Inventario de armas
    for i in range(m):
        model += pulp.lpSum(x[i][j] for j in range(n)) <= w[i]

    # 6) Objetivo
    model += pulp.lpSum(f_j[j] for j in range(n))

    # 7) Resolver con CPLEX (ajusta el path si es necesario)
    model.solve(pulp.CPLEX_CMD(
        path="/Applications/CPLEX_Studio_Community2211/cplex/bin/arm64_osx/cplex",
        msg=False
    ))

    # 8) Leer solución
    sol = [[int(round(pulp.value(x[i][j]))) for j in range(n)] for i in range(m)]
    obj = pulp.value(model.objective)

    return sol, obj, p, V

# —————————————— USO ——————————————
if __name__ == "__main__":
    # Parámetros que puedes cambiar:
    w = [4, 3, 5]   # inventario de armamento de 3 buques
    n = 7           # cantidad de amenazas (blancos)
    K = 10          # refinamos a 10 tramos PWL
    sol, obj, p, V = solve_wta_pwl(w, n, K, seed=218)

    print(f"Resultados Algoritmo Exacto - CPLEX (PWL)")
    print(f"Objetivo (suma supervivencia de la amenaza): {obj:.4f}")
    print("Asignación x[i][j]:")
    for i in range(len(w)):
        for j in range(n):
            if sol[i][j]:
                print(f"  Buque {i} →amenaza {j}: {sol[i][j]} misil(es)")

    # Opcional: imprime p y V para referencia
    print("\np_ij (misil→blanco):")
    for row in p: print(" ", ["{:.2f}".format(v) for v in row])
    print("\nV_j (valor de cada blanco):", ["{:.2f}".format(v) for v in V])


Resultados Algoritmo Exacto - CPLEX (PWL)
Objetivo (suma supervivencia de la amenaza): 9.7666
Asignación x[i][j]:
  Buque 0 →amenaza 2: 1 misil(es)
  Buque 0 →amenaza 4: 1 misil(es)
  Buque 0 →amenaza 5: 2 misil(es)
  Buque 1 →amenaza 0: 1 misil(es)
  Buque 1 →amenaza 6: 2 misil(es)
  Buque 2 →amenaza 1: 2 misil(es)
  Buque 2 →amenaza 3: 2 misil(es)
  Buque 2 →amenaza 4: 1 misil(es)

p_ij (misil→blanco):
  ['0.59', '0.53', '0.78', '0.60', '0.56', '0.77', '0.51']
  ['0.72', '0.60', '0.56', '0.60', '0.57', '0.69', '0.75']
  ['0.64', '0.56', '0.57', '0.64', '0.54', '0.72', '0.54']

V_j (valor de cada blanco): ['7.31', '7.94', '7.31', '6.28', '9.57', '7.58', '9.30']


In [10]:
import math
import random

def solve_wta_greedy(w, n, seed=218):
    random.seed(seed)
    m = len(w)

    # Generar datos igual que el modelo exacto
    p = [[random.uniform(0.5, 0.8) for j in range(n)] for i in range(m)]
    V = [random.uniform(5, 10) for j in range(n)]

    # Calcular q_{ij}
    q = [[1 - p[i][j] for j in range(n)] for i in range(m)]

    # Inicialización
    x = [[0 for j in range(n)] for i in range(m)]
    remaining = w[:]

    # Estrategia greedy
    while True:
        best_delta = float("inf")
        best_i, best_j = None, None

        for i in range(m):
            if remaining[i] == 0:
                continue
            for j in range(n):
                current_yj = sum(x[k][j] * math.log(q[k][j]) for k in range(m))
                current_fj = V[j] * math.exp(current_yj)

                proposed_yj = current_yj + math.log(q[i][j])
                proposed_fj = V[j] * math.exp(proposed_yj)
                delta = proposed_fj - current_fj

                if delta < best_delta:
                    best_delta = delta
                    best_i, best_j = i, j

        if best_i is None:
            break

        x[best_i][best_j] += 1
        remaining[best_i] -= 1

    # Objetivo total
    total_f = 0
    for j in range(n):
        y_j = sum(x[i][j] * math.log(q[i][j]) for i in range(m))
        total_f += V[j] * math.exp(y_j)

    # Mostrar resultados
    print(f"Resultados Heurística - (Greedy)")
    print(f"Objetivo (suma supervivencia de la amenaza): {total_f:.4f}")
    print("Asignación x[i][j]:")
    for i in range(m):
        for j in range(n):
            if x[i][j] > 0:
                print(f"  Buque {i} →amenaza {j}: {x[i][j]} misil(es)")

    print("\np_ij (misil→blanco):")
    for row in p:
        print(" ", ["{:.2f}".format(v) for v in row])

    print("\nV_j (valor de cada blanco):", ["{:.2f}".format(v) for v in V])

# ——— USO ———
if __name__ == "__main__":
    w = [4, 3, 5]
    n = 7
    solve_wta_greedy(w, n, seed=218)


Resultados Heurística - (Greedy)
Objetivo (suma supervivencia de la amenaza): 9.1670
Asignación x[i][j]:
  Buque 0 →amenaza 2: 1 misil(es)
  Buque 0 →amenaza 4: 1 misil(es)
  Buque 0 →amenaza 5: 2 misil(es)
  Buque 1 →amenaza 0: 1 misil(es)
  Buque 1 →amenaza 4: 1 misil(es)
  Buque 1 →amenaza 6: 1 misil(es)
  Buque 2 →amenaza 0: 1 misil(es)
  Buque 2 →amenaza 1: 2 misil(es)
  Buque 2 →amenaza 3: 2 misil(es)

p_ij (misil→blanco):
  ['0.59', '0.53', '0.78', '0.60', '0.56', '0.77', '0.51']
  ['0.72', '0.60', '0.56', '0.60', '0.57', '0.69', '0.75']
  ['0.64', '0.56', '0.57', '0.64', '0.54', '0.72', '0.54']

V_j (valor de cada blanco): ['7.31', '7.94', '7.31', '6.28', '9.57', '7.58', '9.30']
